In [2]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
# all libraries
patientf = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/118_patients/118_patients_bam_vcf.txt'
sailfishf = '/projects/trans_scratch/validations/workspace/szong/Cervical/isoforms/sailfish_files.txt'
mutf = '/projects/trans_scratch/validations/workspace/szong/Cervical/hotspots/rainstorm/mutation_matrix.txt'

In [21]:
df = pd.read_csv(patientf, sep='\t')
df.shape
df.head(5)

(236, 25)

,patient,HIV_status,DNA_lib,source,status,RNA_lib,ribodepleted_lib,DNA_bam,RNA_bam,DNA_single_vcf,...,other_vcf,DNA_tc,RNA_tc,cnv,bbt_transcriptome,bbt_genome,bbt_transcriptome_other_bacterial,bbt_genome_other_bacterial,bbt_transcriptome_other_viral,bbt_genome_other_viral
0,HTMCP-03-06-02002,Negative,A37235,HTMCP_125,Malignant,A37701,A56296,/projects/analysis/analysis22/A37235/merge_bwa...,/projects/analysis/analysis22/IX3432/C67GDANXX...,/projects/analysis/analysis22/A37235/merge_bwa...,...,NaN,70.0,70.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...
1,HTMCP-03-06-02002,Negative,A37160,HTMCP_177,normal,NaN,NaN,/projects/analysis/analysis22/A37160/merge_bwa...,NaN,/projects/analysis/analysis22/A37160/merge_bwa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HTMCP-03-06-02006,Negative,A37239,HTMCP_129,Malignant,A37704,A56298,/projects/analysis/analysis22/A37239/merge_bwa...,/projects/analysis/analysis22/IX3424/C6H58ANXX...,/projects/analysis/analysis22/A37239/merge_bwa...,...,NaN,55.0,55.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...
3,HTMCP-03-06-02006,Negative,A37164,HTMCP_181,normal,NaN,NaN,/projects/analysis/analysis22/A37164/merge_bwa...,NaN,/projects/analysis/analysis22/A37164/merge_bwa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HTMCP-03-06-02008,Negative,A37241,HTMCP_131,Malignant,A37706,NaN,/projects/analysis/analysis22/A37241/merge_bwa...,/projects/analysis/analysis22/IX3430/C67GDANXX...,/projects/analysis/analysis22/A37241/merge_bwa...,...,NaN,50.0,50.0,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...,/projects/NCI_validation2_assembly/NCI_SAIC_HI...


In [22]:
df1 = df[df.status.str.lower() != 'normal']

In [23]:
libs = df1.RNA_lib.tolist()

In [24]:
# read in sailfish files
sdf = pd.read_csv(sailfishf, header=None)
sdf.columns = ['file']
sdf.head()

,file
0,/projects/NCI/NCI_HIV_Cervical/HTMCP-03-06-020...
1,/projects/NCI/NCI_HIV_Cervical/HTMCP-03-06-020...
2,/projects/NCI/NCI_HIV_Cervical/HTMCP-03-06-020...
3,/projects/NCI/NCI_HIV_Cervical/HTMCP-03-06-020...
4,/projects/NCI/NCI_HIV_Cervical/HTMCP-03-06-020...


In [25]:
sdf['lib'] = sdf['file'].apply(lambda x: x.split('/')[6].split('_')[-1])

In [26]:
sdf = sdf[sdf.lib.isin(libs)]

In [27]:
mdf = pd.DataFrame()
for ix, row in sdf.iterrows():
    lib = row.lib
    file = row.file
    pat = file.split('/')[4]
    dftmp = pd.read_csv(file, comment='#', header=None, sep='\t', usecols=[0, 2])
    dftmp.set_index(0, inplace=True)
    dftmp.columns = [pat]
    if mdf.empty:
        mdf = dftmp
    else:
        mdf = pd.merge(mdf, dftmp, left_index=True, right_index=True)
                                        

In [28]:
mdf.head()

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
0,,,,,,,,,,,,,,,,,,,,,
ENST00000553856,33.576200,33.141400,46.404900,77.10720,27.13620,48.014700,30.263100,41.757500,35.221200,36.223200,...,12.21590,6.93319,0.00000,19.07770,62.142400,44.05610,48.322000,48.981600,47.126500,38.511100
ENST00000483851,0.387896,0.018421,0.040748,0.00000,0.00000,0.577957,0.000000,0.568105,0.118517,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.000000,1.10888,0.000000,0.000000,0.880837,0.413576
ENST00000292095,0.000000,0.000000,1.436010,0.00000,0.00000,0.000000,0.000000,0.271039,0.000000,0.523853,...,0.00000,0.00000,0.00000,3.54304,0.000000,1.81668,0.357138,1.924240,4.410340,2.672330
ENST00000361681,339.723000,1473.390000,278.820000,838.59200,667.36800,673.870000,503.075000,467.356000,1521.660000,594.728000,...,243.38500,86.13500,327.41700,112.31900,296.321000,98.90580,76.341200,402.623000,183.403000,120.069000
ENST00000585837,2.549000,1.788040,1.720890,1.45839,2.33313,1.083880,0.064639,2.152970,3.188550,0.160840,...,2.99443,2.02501,1.56969,1.62609,0.062658,1.58455,1.841780,0.172443,5.165530,2.581050


In [29]:
# TERT protein coding isoforms
isoforms = ['ENST00000296820', 'ENST00000508104', 'ENST00000334602', 'ENST00000310581', 'ENST00000460137', 'ENST00000522877', 'ENST00000503656', 'ENST00000484238']
tert = mdf.loc[isoforms,:]
tert

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02036,HTMCP-03-06-02037,...,HTMCP-03-06-02417,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447
0,,,,,,,,,,,,,,,,,,,,,
ENST00000296820,0.030587,0.049220,0.089336,0.0,0.249544,0.624736,0.045502,0.000000,0.032877,0.000000,...,0.069781,0.000000,0.000000,0.028269,0.000000,0.044146,0.000000,1.137000,0.000000,0.000000
ENST00000508104,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000334602,0.000000,0.015010,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.296127,0.000000,0.162024,0.000000,0.139591,0.138158
ENST00000310581,0.098013,0.021332,0.143158,0.0,0.016949,1.704590,0.036992,0.101228,0.170665,0.152239,...,0.000000,0.069270,0.000000,0.000000,0.058421,0.000000,0.041343,0.668444,0.035813,0.061137
ENST00000460137,0.000000,0.047795,0.000000,0.0,0.000000,0.127642,0.000000,0.000000,0.011467,0.000000,...,0.000000,0.000000,0.000000,0.185482,0.000000,0.000000,0.000000,0.238442,0.000000,0.024372
ENST00000522877,0.000000,0.000000,0.000000,0.0,0.000000,0.079057,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000503656,0.000000,0.000000,0.413896,0.0,0.462458,6.638280,0.321471,0.191456,0.510736,0.146406,...,0.000000,0.061922,0.067182,0.918096,1.782080,0.165217,0.000000,5.240340,0.173809,0.219859
ENST00000484238,0.095987,0.210852,0.434865,0.0,0.278292,3.659870,0.088283,0.128414,0.361733,0.193523,...,0.108269,0.055833,0.113863,0.073869,0.675965,0.077069,0.070678,2.593990,0.121068,0.199399


In [30]:
tert.mean(axis=1)

0
ENST00000296820    0.080819
ENST00000508104    0.001564
ENST00000334602    0.064021
ENST00000310581    0.511876
ENST00000460137    0.026368
ENST00000522877    0.014821
ENST00000503656    1.663364
ENST00000484238    1.113691
dtype: float64

In [31]:
tert.to_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/isoforms/isoforms_neg.txt')

In [5]:
muts = pd.read_csv(mutf, low_memory=False)
muts.head()

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Variant_Type,Tumor_Sample_Barcode,IMPACT,patient
0,OR4G11P,1,59734,59734,5'Flank,SNP,HTMCP-03-06-02001_T,MODIFIER,HTMCP-03-06-02001
1,RP11-206L10.5,1,694016,694016,3'Flank,SNP,HTMCP-03-06-02001_T,MODIFIER,HTMCP-03-06-02001
2,AL669831.1,1,737454,737454,3'Flank,SNP,HTMCP-03-06-02001_T,MODIFIER,HTMCP-03-06-02001
3,FAM87B,1,757477,757477,3'Flank,SNP,HTMCP-03-06-02001_T,MODIFIER,HTMCP-03-06-02001
4,LINC01128,1,758253,758253,5'Flank,SNP,HTMCP-03-06-02001_T,MODIFIER,HTMCP-03-06-02001


In [6]:
muts_tert = muts[muts.Hugo_Symbol == 'TERT']

In [7]:
muts_tert

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Variant_Type,Tumor_Sample_Barcode,IMPACT,patient
7881,TERT,5,1291803,1291803,Intron,SNP,HTMCP-03-06-02001_T,MODIFIER,HTMCP-03-06-02001
54576,TERT,5,1284161,1284161,Intron,SNP,HTMCP-03-06-02006_T,MODIFIER,HTMCP-03-06-02006
89302,TERT,5,1249821,1249821,3'Flank,SNP,HTMCP-03-06-02012_T,MODIFIER,HTMCP-03-06-02012
89303,TERT,5,1291075,1291075,Intron,SNP,HTMCP-03-06-02012_T,MODIFIER,HTMCP-03-06-02012
89304,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02012_T,MODIFIER,HTMCP-03-06-02012
133199,TERT,5,1284668,1284668,Intron,SNP,HTMCP-03-06-02020_T,MODIFIER,HTMCP-03-06-02020
167913,TERT,5,1274248,1274248,Intron,SNP,HTMCP-03-06-02040_T,MODIFIER,HTMCP-03-06-02040
167914,TERT,5,1276186,1276186,Intron,SNP,HTMCP-03-06-02040_T,MODIFIER,HTMCP-03-06-02040
285926,TERT,5,1249895,1249895,3'Flank,SNP,HTMCP-03-06-02058_T,MODIFIER,HTMCP-03-06-02058
285927,TERT,5,1253347,1253347,3'UTR,SNP,HTMCP-03-06-02058_T,MODIFIER,HTMCP-03-06-02058


In [17]:
muts_tert[(muts_tert.Start_Position >=1283266)&(muts_tert.Start_Position<=1284668)]['patient'].count()#.nunique()
muts_tert[(muts_tert.Start_Position >=1295228)&(muts_tert.Start_Position<=1295250)]#['patient'].count()#nunique()


23

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Variant_Type,Tumor_Sample_Barcode,IMPACT,patient
89304,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02012_T,MODIFIER,HTMCP-03-06-02012
863963,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02120_T,MODIFIER,HTMCP-03-06-02120
966501,TERT,5,1295250,1295250,5'Flank,SNP,HTMCP-03-06-02148_T,MODIFIER,HTMCP-03-06-02148
1153559,TERT,5,1295250,1295250,5'Flank,SNP,HTMCP-03-06-02176_T,MODIFIER,HTMCP-03-06-02176
1596626,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02219_T,MODIFIER,HTMCP-03-06-02219
1631394,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02225_T,MODIFIER,HTMCP-03-06-02225
1748207,TERT,5,1295250,1295250,5'Flank,SNP,HTMCP-03-06-02259_T,MODIFIER,HTMCP-03-06-02259
1887642,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02268_T,MODIFIER,HTMCP-03-06-02268
1936483,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02326_T,MODIFIER,HTMCP-03-06-02326
1964012,TERT,5,1295228,1295228,5'Flank,SNP,HTMCP-03-06-02328_T,MODIFIER,HTMCP-03-06-02328
